# POLI/USP Trabalho de Conclusão de Curso
## Fine Tuning - Modelo Bode
Autor: Gabriel Benvegmi

Esse notebook faz o processo de Fine-tuning para o modelo Bode. Ele foi executado no Google Colab.

In [ ]:
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q loralib
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q einops accelerate bitsandbytes
!pip install -q sentence_transformers
!pip install -q git+https://github.com/huggingface/peft.git


In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512


In [ ]:
import json
import os

import pandas as pd
from datasets import load_dataset, Dataset
from evaluate import load
from google.colab import drive, userdata
from huggingface_hub import login
from peft import get_peft_model, prepare_model_for_int8_training, prepare_model_for_kbit_training, LoraConfig, PeftModel, PeftConfig
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

In [ ]:
metric = load("rouge")

In [ ]:
os.getenv("PYTORCH_CUDA_ALLOC_CONF")

'max_split_size_mb:512'

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
login(userdata.get('huggingface_token'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Carregando modelo

In [ ]:
save_path = "/content/drive/MyDrive/Estudos/Poli/TCC/notebooks/"

In [ ]:
llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br'
hf_auth = userdata.get('huggingface_token')
config = PeftConfig.from_pretrained(llm_model)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto', token=hf_auth, cache_dir=save_path + "cache")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, token=hf_auth, cache_dir=save_path + "cache")
model = PeftModel.from_pretrained(model, llm_model) # Caso ocorra o seguinte erro: "ValueError: We need an `offload_dir`... Você deve acrescentar o parâmetro: offload_folder="./offload_dir".

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [ ]:
# Testando geração de texto, conforme instruído na página original do modelo de
# referência
def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Entrada:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:"""

generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    num_beams=2,
    do_sample=True
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_length=1000
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Resposta:", output.split("### Resposta:")[1].strip())

In [ ]:
evaluate("Responda com detalhes: O que é um bode?")

Resposta: Um bode é um animal de pequeno porte, geralmente de cor marrom ou cinza, com uma cauda longa e uma cabeça grande. Eles são conhecidos por sua habilidade de correr rapidamente e podem atingir velocidades de até 45 milhas por hora. Eles são onívoros e se alimentam de uma variedade de alimentos, incluindo grãos, frutas, raízes e insetos.</s>


## Carregando dados para fine-tuning

In [ ]:
def generate_prompt(input, output=None):
  prompt = f"""# CONTEXTO #
Extrair as informações principais de documentos do ecossistema financeiro.

# OBJETIVO #
Sumarize adequadamente o documento presente na ENTRADA. Extraia até três passagens principais do texto original, evitando ao máximo paráfrases, se atendo ao conteúdo literal.

# ESTILO #
Siga o estilo direto e impessoal dos documentos da ENTRADA. Evite enumerações ou a numeração das sentenças de saída, apenas concatene as sentenças em um formato comum de texto contínuo em sua ordem original.

# TOM #
Técnico.

# ENTRADA #
{input}

# RESPOSTA #
Sumário: """
  if output is not None:
     prompt += output

  return prompt

def evaluate(input):
    prompt = generate_prompt(input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_length=100
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Resposta:\n" + output.split("# RESPOSTA #\nSumário:")[1].strip())

In [ ]:
docs_df = pd.read_csv(save_path + "2_summaries_dataset.csv", index_col=0, usecols=[1, 2, 3])

In [ ]:
df_train, df_val = train_test_split(docs_df, shuffle=True, random_state=42, test_size=0.15)
print(df_train.shape)
print(df_val.shape)

(60, 2)
(11, 2)


In [ ]:
df_train.head()

,source_text,target_summary
published_title,,
100-2023-VNC-Comunicado Externo,"Informamos que, em 30/10/2023, será implementa...","Informamos que, em 30/10/2023, será implementa..."
104-2023-PRE-Ofício Circular,"Informamos que, em 30/06/2023, entrará em vigo...","Informamos que, em 30/06/2023, entrará em vigo..."
104-2023-VNC-Comunicado Externo,"A B3 informa que, a partir do dia 08/11/2023, ...","A B3 informa que, a partir do dia 08/11/2023, ..."
036-2023-VPC-Comunicado Externo,"Informamos que, nos dias 22 e 29/07/2023, será...","Informamos que, nos dias 22 e 29/07/2023, será..."
079-2023-VNC-Comunicado Externo,Conforme divulgado no item 4.3.3.1 do Manual d...,Conforme divulgado no item 4.3.3.1 do Manual d...


In [ ]:
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

In [ ]:
if tokenizer.pad_token is None:
  print("Adicionando PAD Token...")
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))
else:
  print(f"Pad Token  ----> {tokenizer.pad_token}")

Adicionando PAD Token...


In [ ]:
proc_train_data = train_dataset.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point['source_text'], data_point['target_summary']),
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
)

proc_val_data = val_dataset.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point['source_text'], data_point['target_summary']),
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

## Configuração do Modelo

In [ ]:
# Ver https://stackoverflow.com/a/76262549
config.inference_mode = False
# config.base_model_name_or_path='meta-llama/Llama-2-7b-chat-hf'

In [ ]:
# https://discuss.huggingface.co/t/peft-lora-gpt-neox-backward-pass-failing/35641/2

if hasattr(model, "enable_input_require_grads"):
  print("Habilitando...")
  model.enable_input_require_grads()
else:
  print("Registrando forward hook")
  def make_inputs_require_grad(module, input, output):
        output.requires_grad_(True)

  model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

Enabling...


In [ ]:
# Antes de aplicar a configuração do LORA...
model.print_trainable_parameters()

trainable params: 262,152,192 || all params: 6,742,618,112 || trainable%: 3.8879881322870924


In [ ]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = True)
model = get_peft_model(model, config)

In [ ]:
# ...Depois de aplicar a configuração.
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,618,112 || trainable%: 0.06220586618327525


In [ ]:
import nltk
import numpy as np

# ver https://discuss.huggingface.co/t/calculating-rouge-metric-for-fine-tunning-pegasus/6436
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Adafactor, get_cosine_schedule_with_warmup

learning_rate = 1e-3
warmup_steps = 50

batch_size = 4
epochs = 100
total_steps = round(len(proc_train_data) / batch_size) * epochs
save_steps = total_steps // 5
beta1 = 0.9

print(
    "batch_size  ", batch_size, "\n"
    "epochs      ", epochs, "\n"
    "total_steps ", total_steps, "\n"
    "beta1       ", beta1
)

optimizer = Adafactor(
    model.parameters(),
    lr=learning_rate,
    beta1=beta1,
    weight_decay=0,  # Set weight decay to 0 since dynamic weight decay is applied separately
    scale_parameter=True,
    relative_step=False
)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps,
)


training_args = TrainingArguments(
    output_dir=save_path + "bode_checkpoints/b3-summ-bode-7b-alpaca-pt-br",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=save_steps,  # Adjust as needed
    save_total_limit=9,  # Adjust as needed
    learning_rate=learning_rate,
    weight_decay=0.0,
    warmup_steps=warmup_steps,
    fp16=True,
)

batch_size   4 
epochs       100 
total_steps  1500 
beta1        0.9


In [ ]:
data_collator_lm = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    train_dataset=proc_train_data, #proc_data["train"],
    eval_dataset=proc_val_data,
    args=training_args,
    optimizers=(optimizer, scheduler),
    data_collator=data_collator_lm,
)

# Desabilitando cache por incompatibilidade com gradient accumulation
model.config.use_cache = False

In [ ]:
trainer.train(resume_from_checkpoint=False)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.554400
2,2.616500
3,2.623600
4,2.606400
5,2.564200
6,2.595800
7,2.591900
8,2.577000
9,2.598700
10,2.556000


TrainOutput(global_step=20, training_loss=2.5660656213760378, metrics={'train_runtime': 2033.9698, 'train_samples_per_second': 0.349, 'train_steps_per_second': 0.01, 'total_flos': 1.2836335270232064e+16, 'train_loss': 2.5660656213760378, 'epoch': 8.89})

In [ ]:
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gbieul/lora-alpaca/commit/17b4796e2d90d114b871d5821d4b37fc0c35c722', commit_message='End of training', commit_description='', oid='17b4796e2d90d114b871d5821d4b37fc0c35c722', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("gbieul/b3-summ-bode-7b-alpaca-pt-br")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/gbieul/b3-summ-bode-7b-alpaca-pt-br/commit/7b46d31eaacfe351bff78370736894b44a0a710f', commit_message='Upload tokenizer', commit_description='', oid='7b46d31eaacfe351bff78370736894b44a0a710f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("gbieul/b3-summ-bode-7b-alpaca-pt-br")

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gbieul/b3-summ-bode-7b-alpaca-pt-br/commit/9e4ec0d1c5400dd862390bf92c7ad75a31b08900', commit_message='Upload model', commit_description='', oid='9e4ec0d1c5400dd862390bf92c7ad75a31b08900', pr_url=None, pr_revision=None, pr_num=None)

## Geração

In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [ ]:
llm_model = 'gbieul/b3-summ-bode-7b-alpaca-pt-br'
hf_auth = userdata.get('huggingface_token')

config = PeftConfig.from_pretrained(llm_model)
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf', trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto', token=hf_auth, cache_dir=save_path + "cache")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf', token=hf_auth, cache_dir=save_path + "cache")
peft_model = PeftModel.from_pretrained(model, llm_model)

# Ver https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo
peft_model = peft_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:74: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
if tokenizer.pad_token is None:
  print("Adicionando PAD Token...")
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  peft_model.resize_token_embeddings(len(tokenizer))
else:
  print(f"Pad Token  ----> {tokenizer.pad_token}")

Adicionando PAD Token...


In [ ]:
generation_config = GenerationConfig(
    # temperature=0.2,
    # top_p=0.75,
    max_length=256,
    num_beams=2,
    repetition_penalty=2.5,
    length_penalty=-5.0,
    early_stopping=True
)

In [ ]:
max_length = 1024

def generate_summaries(input):
  prompt = generate_prompt(input)
  tokenized = tokenizer(
              prompt,
              return_tensors="pt",
              truncation=True,
              max_length=max_length-1,
              padding="max_length",
          )
  input_ids = tokenized.input_ids.cuda()
  mask = tokenized.attention_mask.cuda()

  outputs = peft_model.generate(
              input_ids=input_ids,
              attention_mask=mask,
              max_length=max_length,
              max_new_tokens=256,
              num_beams=2,
              generation_config=generation_config,
              repetition_penalty=2.5,
              length_penalty=-5.0,
              early_stopping=True,
          )

  generated_chunk = tokenizer.decode(
      outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
  )

  return generated_chunk

In [ ]:
docs_df.head()

,source_text,target_summary
published_title,,
187-2023-PRE-Ofício Circular,"Informamos que, a partir de 13/11/2023, inclus...","Informamos que, a partir de 13/11/2023, inclus..."
107-2023-VNC-Comunicado Externo,"A B3 informa que, conforme Comunicado Externo ...","A B3 informa que, conforme Comunicado Externo ..."
059-2023-VPC-Comunicado Externo,"A B3 informa que, no dia 13/11/2023, atualizar...","A B3 informa que, no dia 13/11/2023, atualizar..."
186-2023-PRE-Ofício Circular,"Informamos que, a partir de 27/11/2023, serão ...","Informamos que, a partir de 27/11/2023, serão ..."
184-2023-PRE-Ofício Circular,"Informamos que, em 21/11/2023, entrarão em vig...","Informamos que, em 21/11/2023, entrarão em vig..."


In [ ]:
docs_df["raw_summary"] = docs_df["source_text"].progress_apply(generate_summaries)

100%|██████████| 71/71 [51:51<00:00, 43.83s/it]


In [ ]:
docs_df.to_csv(save_path + "3_bode_raw_summaries_generation.csv")

In [ ]:
# Separando o conteúdo gerado do prompt original
docs_df["generated_summary"] = docs_df["raw_summary"].progress_apply(lambda row: row.split("# RESPOSTA #\nSumário:")[-1].strip())

100%|██████████| 71/71 [00:00<00:00, 44493.59it/s]


In [ ]:
docs_df.head()

,source_text,target_summary,raw_summary,generated_summary
published_title,,,,
187-2023-PRE-Ofício Circular,"Informamos que, a partir de 13/11/2023, inclus...","Informamos que, a partir de 13/11/2023, inclus...",# CONTEXTO #\nExtrair as informações principai...,
107-2023-VNC-Comunicado Externo,"A B3 informa que, conforme Comunicado Externo ...","A B3 informa que, conforme Comunicado Externo ...",# CONTEXTO #\nExtrair as informações principai...,", desde 08/11/2023, são disponíveis em ambient..."
059-2023-VPC-Comunicado Externo,"A B3 informa que, no dia 13/11/2023, atualizar...","A B3 informa que, no dia 13/11/2023, atualizar...",# CONTEXTO #\nExtrair as informações principai...,* A B3 irá atualizar os dados do cadastro das ...
186-2023-PRE-Ofício Circular,"Informamos que, a partir de 27/11/2023, serão ...","Informamos que, a partir de 27/11/2023, serão ...",# CONTEXTO #\nExtrair as informações principai...,", a partir de 27/11/2023, serão implementados ..."
184-2023-PRE-Ofício Circular,"Informamos que, em 21/11/2023, entrarão em vig...","Informamos que, em 21/11/2023, entrarão em vig...",# CONTEXTO #\nExtrair as informações principai...,1. Novas versões do Regulamento da Câmara B3 e...


In [ ]:
docs_df["rouge_scores"] = docs_df.progress_apply(lambda row: metric.compute(predictions=[row["generated_summary"]], references=[row["target_summary"]]), axis=1)

100%|██████████| 71/71 [00:14<00:00,  4.96it/s]


In [ ]:
scores = docs_df["rouge_scores"].apply(pd.Series)
docs_with_rouge = docs_df.drop(columns="rouge_scores").merge(scores, left_index=True, right_index=True)

In [ ]:
docs_df.to_csv(save_path + "3_bode_summaries.csv")

In [ ]:
import textwrap
import pandas as pd

In [ ]:
save_path = "/content/drive/MyDrive/Estudos/Poli/TCC/notebooks/"
docs_with_rouge = pd.read_csv(save_path + "3_bode_summaries.csv", index_col=0)
docs_with_rouge.head()

,source_text,target_summary,raw_summary,generated_summary,rouge1,rouge2,rougeL,rougeLsum
published_title,,,,,,,,
187-2023-PRE-Ofício Circular,"Informamos que, a partir de 13/11/2023, inclus...","Informamos que, a partir de 13/11/2023, inclus...",# CONTEXTO #\nExtrair as informações principai...,NaN,0.000000,0.000000,0.000000,0.000000
107-2023-VNC-Comunicado Externo,"A B3 informa que, conforme Comunicado Externo ...","A B3 informa que, conforme Comunicado Externo ...",# CONTEXTO #\nExtrair as informações principai...,", desde 08/11/2023, são disponíveis em ambient...",0.744361,0.696970,0.744361,0.744361
059-2023-VPC-Comunicado Externo,"A B3 informa que, no dia 13/11/2023, atualizar...","A B3 informa que, no dia 13/11/2023, atualizar...",# CONTEXTO #\nExtrair as informações principai...,* A B3 irá atualizar os dados do cadastro das ...,0.544379,0.395210,0.508876,0.473373
186-2023-PRE-Ofício Circular,"Informamos que, a partir de 27/11/2023, serão ...","Informamos que, a partir de 27/11/2023, serão ...",# CONTEXTO #\nExtrair as informações principai...,", a partir de 27/11/2023, serão implementados ...",0.669323,0.546185,0.605578,0.605578
184-2023-PRE-Ofício Circular,"Informamos que, em 21/11/2023, entrarão em vig...","Informamos que, em 21/11/2023, entrarão em vig...",# CONTEXTO #\nExtrair as informações principai...,1. Novas versões do Regulamento da Câmara B3 e...,0.507177,0.386473,0.411483,0.468900


In [ ]:
for title, row in docs_with_rouge.head(10).iterrows():
  print(title)
  print(f"Scores: 'rouge1' {row['rouge1']: .2f} | 'rouge2' {row['rouge2']: .2f} | 'rougeL' {row['rougeL']: .2f} | 'rougeLsum' {row['rougeLsum']: .2f}")
  print("###### Original")
  print(textwrap.fill(row["source_text"], 70), "\n\n")
  print("###### Resumo referência")
  print(textwrap.fill(row["target_summary"], 70))
  print("###### Resumo gerado")
  print(textwrap.fill(str(row["generated_summary"]), 70))
  print("\n\n\n")

187-2023-PRE-Ofício Circular
Scores: 'rouge1'  0.00 | 'rouge2'  0.00 | 'rougeL'  0.00 | 'rougeLsum'  0.00
###### Original
Informamos que, a partir de 13/11/2023, inclusive, será alterada a
regra para cadastro automático de vencimentos do Contrato Futuro de
Cupom de IPCA (DAP). A regra de cadastro automático de vencimentos foi
revista, visando ampliar a abertura de novos vencimentos curtos de
DAP, de 3 primeiros meses para 6, conforme tabela abaixo. Regra até
10/11/2023. Regra a partir de 13/11/2023. A regra vigente de cadastro
automático e cadastro pré-aprovado de instrumentos para o DAP pode ser
consultada em Regulação, Estrutura normativa, Regulamentos e manuais,
Negociação, Acessar documentos, Critérios para Criação e Exclusão de
Vencimentos Futuros e Séries de Opções. Ressaltamos que os demais
termos do Contrato Futuro de Cupom de IPCA (DAP) permanecem
inalterados. O único ajuste é referente ao cadastro automático. 


###### Resumo referência
Informamos que, a partir de 13/11/2023,